In [4]:
import torch
from PIL import Image
from torchvision import transforms
Image.MAX_IMAGE_PIXELS = None

def read_img_file(f):
    img = Image.open(f)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    return img

_transform_test_random=transforms.Compose([
        transforms.RandomCrop((256,256)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

_transform_test_random_vit = transforms.Compose([
        transforms.RandomCrop((252,252)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [5]:
model_type = "eva" # convnext effnet

if model_type=="eva":
    _transform = _transform_test_random_vit
    model = torch.load("./models/model_eva.pth")
elif model_type=="convnext":
    _transform = _transform_test_random
    model = torch.load("./models/model_convnext.pth")
else:
    _transform = _transform_test_random
    model = torch.load("./models/model_effnet.pth")
model = model.eval()

In [6]:
from statistics import mean

# 10 try method
with torch.inference_mode():
    img = read_img_file('./test.jpg')
    with torch.inference_mode():
        tmp=[]
        for _ in range(10):
            img_random_crop = _transform(img).cuda()
            outputs = model.forward(img_random_crop.unsqueeze(0))
            outputs = torch.sigmoid(outputs).cpu().numpy()
            tmp.append(outputs[0][0])
        print(tmp)
        print("10 try method: ", mean(tmp))

#1 try method
with torch.inference_mode():
    img = read_img_file('./test.jpg')
    with torch.inference_mode():
        img_crop = _transform(img).cuda()
        outputs = model.forward(img_crop.unsqueeze(0))
        outputs = torch.sigmoid(outputs).cpu().numpy()
        print("1 try method: ",outputs)

[0.9999869, 0.9999893, 0.99989045, 0.99998987, 0.99998975, 0.9999895, 0.9999901, 0.99999, 0.99993503, 0.99998915]
10 try method:  0.999974
1 try method:  [[0.9895235]]
